## Background

For our analysis, we will use pandas_datareader to import stock market data from Yahoo Finance. Import some popular companies as well as invested ones such as CGI, Apple, Microsoft, Alphabet (Google), Amazon, Tesla, Meta (Facebook).


[Reference](https://medium.datadriveninvestor.com/stock-market-analysis-with-python-plotly-dash-and-powerbi-48975c48fe73)

---

## Data Acquisition

In [1]:
#!pip install pandas_datareader pandas 


     |████████████████████████████████| 109 kB 329 kB/s eta 0:00:01


In [1]:
import pandas_datareader as data
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta

In [2]:
my_companies_code = ['GIB', 'AAPL', 'MSFT', 'GOOG', 'TSLA', 'FB']
my_companies_name = ['CGI', 'Apple', 'Microsoft', 'Google', 'Tesla', 'Facebook']

In [3]:
# Scrape ten largest tech companies by market capitalization
html = requests.get('https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/').text
soup = BeautifulSoup(html, 'html.parser')
company_name = [x.get_text(strip=True) for x in soup.find_all('div', {'class': 'company-name'})][:10]
company_code = [x.get_text(strip=True) for x in soup.find_all('div', {'class': 'company-code'})][:10]

In [4]:
company_name

['Apple',
 'Microsoft',
 'Alphabet (Google)',
 'Amazon',
 'Tesla',
 'Meta (Facebook)',
 'TSMC',
 'Tencent',
 'NVIDIA',
 'Samsung']

In [5]:
company_code

['AAPL',
 'MSFT',
 'GOOG',
 'AMZN',
 'TSLA',
 'FB',
 'TSM',
 'TCEHY',
 'NVDA',
 '005930.KS']

In [6]:
now = datetime.now()
data_source = 'yahoo'
start_date = now - relativedelta(years=5)
end_date = now
dataArray = []
for x, y in zip(my_companies_name,my_companies_code):
    d = data.DataReader(y, data_source, start_date, end_date)
    d['stockName'] = x
    d['stockCode'] = y
    dataArray.append(d)
df = pd.concat(dataArray)
df.reset_index(inplace=True)
print(df)

           Date        High         Low        Open       Close      Volume  \
0    2017-05-15   49.169998   48.580002   48.580002   48.970001    209100.0   
1    2017-05-16   49.380001   48.980000   48.980000   49.259998    201000.0   
2    2017-05-17   48.950001   47.970001   48.709999   48.000000    246300.0   
3    2017-05-18   48.480000   47.750000   47.799999   48.080002    182500.0   
4    2017-05-19   48.880001   48.160000   48.220001   48.700001    188400.0   
...         ...         ...         ...         ...         ...         ...   
7555 2022-05-09  202.570007  195.580002  199.839996  196.210007  36303200.0   
7556 2022-05-10  202.960007  194.190002  199.509995  197.649994  34452500.0   
7557 2022-05-11  200.669998  188.270004  196.330002  188.740005  31153400.0   
7558 2022-05-12  198.050003  184.779999  187.059998  191.240005  38262200.0   
7559 2022-05-13  199.889999  191.279999  192.580002  198.619995  24523500.0   

       Adj Close stockName stockCode  
0      48.97

In [7]:
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close,stockName,stockCode
0,2017-05-15,49.169998,48.580002,48.580002,48.970001,209100.0,48.970001,CGI,GIB
1,2017-05-16,49.380001,48.980000,48.980000,49.259998,201000.0,49.259998,CGI,GIB
2,2017-05-17,48.950001,47.970001,48.709999,48.000000,246300.0,48.000000,CGI,GIB
3,2017-05-18,48.480000,47.750000,47.799999,48.080002,182500.0,48.080002,CGI,GIB
4,2017-05-19,48.880001,48.160000,48.220001,48.700001,188400.0,48.700001,CGI,GIB


---
## Data Visualization

In [8]:
df_clone = df.copy()
df_clone.set_index('Date')
fig =px.line(df_clone[df_clone['stockCode']=='GIB']['Close'])
fig.update_yaxes(title_text='')
fig.update_traces(line_color='#FF0000')
fig.show()

<IPython.core.display.Javascript object>

In [8]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go

app = dash.Dash()   

def stock_prices():
    fig = go.Figure([go.Scatter(x = df[df['stockCode']=='GIB']['Date'], y = df[df['stockCode']=='GIB']['Close'],\
                     line = dict(color = 'firebrick', width = 4), name = 'CGI')
                     ])
    fig.update_layout(title = 'Closed Prices over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Prices'
                      )
    return fig


app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'CGI Stock Market Analysis', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),


        dcc.Graph(id = 'line_plot', figure = stock_prices())
    ]
                     )

app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_11161/2661170204.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_11161/2661170204.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/May/2022 22:41:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 22:41:07] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1635345671.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 22:41:07] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1635345671.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 22:41:07] "GET /_dash-component-suites/dash/deps/react-dom@16.v2

In [9]:
now = datetime.now()
data_source = 'yahoo'
start_date = now - relativedelta(years=5)
end_date = now
dataArray = []
for x, y in zip(my_companies_name,my_companies_code):
    d = data.DataReader(y, data_source, start_date, end_date)
    dataArray.append(d)
df = pd.concat(dataArray, axis=1, keys=my_companies_name)

Drop google for scale

In [10]:
df.drop('Google',axis=1, inplace=True)


Drop Tesla


In [11]:
df.drop('Tesla',axis=1, inplace=True)


In [ ]:
import matplotlib.pyplot as plt

df.columns.names = ['Stock Ticker', 'Stock Info']
close = df.xs(key='Close', axis=1, level='Stock Info')
plt.figure(figsize=(12, 6))

fig = px.area(close, facet_col='Stock Ticker', facet_col_wrap=2)
fig.show()

<IPython.core.display.Javascript object>

<Figure size 864x432 with 0 Axes>